<a href="https://colab.research.google.com/github/enelene/ML-assignment-1/blob/main/model_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation of required libraries and env variables


In [2]:
!pip install mlflow dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import mlflow
from dagshub import dagshub_logger


In [4]:
import os
os.environ["MLFLOW_TRACKING_USERNAME"] = "enelene"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "8gW@uZ@KYuCwFuU"


In [5]:
# Replace <username> and <repo_name> with your DagsHub info
mlflow.set_tracking_uri("https://dagshub.com/enelene/ML-assignment-1.mlflow")
mlflow.set_experiment("house-prices-experiment")


<Experiment: artifact_location='mlflow-artifacts:/17fe90981cbc4a7aaeaf700c3b4fd397', creation_time=1743927842431, experiment_id='0', last_update_time=1743927842431, lifecycle_stage='active', name='house-prices-experiment', tags={}>

In [6]:
!pip install kaggle


In [7]:
from google.colab import files
files.upload()  # Then select your kaggle.json file


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"elenegabeskiria","key":"0170843dd939206cd157d6cdb2b13816"}'}

In [8]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [9]:
!kaggle competitions download -c house-prices-advanced-regression-techniques


In [10]:
!unzip house-prices-advanced-regression-techniques.zip


Archive:  house-prices-advanced-regression-techniques.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [11]:
import os
import pandas as pd
import numpy as np

# ML libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

# MLflow libraries
import mlflow
import mlflow.sklearn

In [12]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Keep a backup just in case
train_original = train_df.copy()
test_original = test_df.copy()

In [13]:
train_df.info()
train_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [14]:
print("Train shape:", train_df.shape)
print("Test shape :", test_df.shape)

Train shape: (1460, 81)
Test shape : (1459, 80)


In [24]:
def fill_missing_values(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].fillna('None')
        elif pd.api.types.is_numeric_dtype(df[col]):
            # Fill numeric columns with the median
            # df[col] = df[col].fillna(df[col].median())
            # Or fill with 0
            df[col] = df[col].fillna(0)
    return df

def show_missing_values(df):
    missing_values = df.isnull().sum()
    missing_values_percentage = (missing_values / len(df)) * 100
    missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_values_percentage})
    missing_data = missing_data[missing_data['Missing Values'] > 0]
    missing_data = missing_data.sort_values('Percentage', ascending=False)
    return missing_data


In [23]:
print(show_missing_values(test_original))
print(show_missing_values(train_original))

              Missing Values  Percentage
PoolQC                  1456   99.794380
MiscFeature             1408   96.504455
Alley                   1352   92.666210
Fence                   1169   80.123372
MasVnrType               894   61.274846
FireplaceQu              730   50.034270
LotFrontage              227   15.558602
GarageQual                78    5.346127
GarageCond                78    5.346127
GarageYrBlt               78    5.346127
GarageFinish              78    5.346127
GarageType                76    5.209047
BsmtCond                  45    3.084304
BsmtQual                  44    3.015764
BsmtExposure              44    3.015764
BsmtFinType1              42    2.878684
BsmtFinType2              42    2.878684
MasVnrArea                15    1.028101
MSZoning                   4    0.274160
Functional                 2    0.137080
BsmtFullBath               2    0.137080
Utilities                  2    0.137080
BsmtHalfBath               2    0.137080
Exterior1st     

In [ ]:
train_df = fill_missing_values(train_df)
test_df = fill_missing_values(test_df)

print("Missing values in train dataset:")
print(show_missing_values(train_df))
print("\nMissing values in test dataset:")
print(show_missing_values(test_df))

In [21]:
SEED = 42
np.random.seed(SEED)

In [25]:
train_df["TotalSF"] = train_df["TotalBsmtSF"] + train_df["1stFlrSF"] + train_df["2ndFlrSF"]
test_df["TotalSF"] = test_df["TotalBsmtSF"] + test_df["1stFlrSF"] + test_df["2ndFlrSF"]


In [26]:
y = np.log1p(train_df["SalePrice"])

In [27]:
train_df.drop(["Id","SalePrice"], axis=1, inplace=True)
test_ids = test_df["Id"]
test_df.drop("Id", axis=1, inplace=True)

In [28]:
all_data = pd.concat([train_df, test_df], axis=0)  # Combine to ensure same dummy columns
all_data = pd.get_dummies(all_data, drop_first=True)


In [29]:
train_transformed = all_data.iloc[:len(train_df), :]
test_transformed = all_data.iloc[len(train_df):, :]

In [30]:
X = train_transformed.values
X_test = test_transformed.values

In [31]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

In [48]:
def train_and_log(model, model_name, params=None):
    with mlflow.start_run(run_name=model_name):
        if params:
            for p_name, p_val in params.items():
                mlflow.log_param(p_name, p_val)

        # Fit and evaluate
        model.fit(X_train, y_train)
        preds_val = model.predict(X_val)
        rmse_val = np.sqrt(mean_squared_error(y_val, preds_val))

        mlflow.log_metric("rmse_val", rmse_val)

        # Log model artifact
        mlflow.sklearn.log_model(model, artifact_path="model")

        print(f"{model_name} RMSE (Val): {rmse_val:.4f}")
        return rmse_val, model

In [33]:
# 7.1 Linear Regression (Baseline)
linreg_params = {}
train_and_log(LinearRegression(), "LinearRegression", linreg_params)

# 7.4 Random Forest
rf_params = {"n_estimators": 100, "max_depth": 10}
rf_model = RandomForestRegressor(
    n_estimators=rf_params["n_estimators"],
    max_depth=rf_params["max_depth"],
    random_state=SEED
)
train_and_log(rf_model, "RandomForest", rf_params)

2025/04/09 07:45:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


LinearRegression RMSE (Val): 0.1734
🏃 View run LinearRegression at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0/runs/c6262b87208f45efaefd25decf7d2688
🧪 View experiment at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0


2025/04/09 07:45:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RandomForest RMSE (Val): 0.1508
🏃 View run RandomForest at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0/runs/4d08c49fede34668a668d46b5dec13f3
🧪 View experiment at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0


In [34]:
all_data.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_None,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,...,False,False,False,False,True,False,False,False,True,False
1,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,...,False,False,False,False,True,False,False,False,True,False
2,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,...,False,False,False,False,True,False,False,False,True,False
3,70,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,...,False,False,False,False,True,False,False,False,False,False
4,60,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,...,False,False,False,False,True,False,False,False,True,False


In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Suppose X, y are your features/target for training (e.g. after preprocessing)
# X_train, X_val, y_train, y_val is your manual split, or you might skip and rely purely on CV

# 1. Create a base RandomForestRegressor (without specifying n_estimators or max_depth)
rf = RandomForestRegressor(random_state=SEED)

# 2. Define the parameter grid you want to search
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [5, 10, 15, None]
}

# 3. Set up the GridSearchCV
#    - scoring="neg_mean_squared_error" is good for regression
#    - cv=5 means 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring="neg_mean_squared_error",  # scikit-learn uses negative MSE for max objective
    cv=5,                              # 5 folds
    verbose=2,                         # print logs while training (optional)
    n_jobs=-1                          # use all available CPU cores
)

# 4. Fit the grid search on your whole training set
grid_search.fit(X_train, y_train)

# 5. Get the best parameters and best estimator
print("Best params found:", grid_search.best_params_)
print("Best CV score:", grid_search.best_score_)

best_rf = grid_search.best_estimator_

# 6. Evaluate on your validation set (or do cross_val_predict)
y_val_preds = best_rf.predict(X_val)
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_preds))

print("Validation RMSE with best params:", val_rmse)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best params found: {'max_depth': None, 'n_estimators': 200}
Best CV score: -0.021024343017266335
Validation RMSE with best params: 0.14754298333238972


In [36]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Example: define distributions for random sampling
param_dist = {
    "n_estimators": randint(50, 500),  # random int between 50 and 499
    "max_depth": [5, 10, 15, None]
}

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    scoring="neg_mean_squared_error",
    cv=5,
    n_iter=10,  # how many random combos to try
    random_state=SEED
)
random_search.fit(X_train, y_train)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   param_distributions={'max_depth': [5, 10, 15, None],
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a463c56dfd0>},
                   random_state=42, scoring='neg_mean_squared_error')

In [37]:
rf_params = {"n_estimators": 200, "max_depth": None }
rf_model = RandomForestRegressor(
    n_estimators=rf_params["n_estimators"],
    max_depth=rf_params["max_depth"],
    random_state=SEED
)
train_and_log(rf_model, "RandomForest", rf_params)

2025/04/09 08:55:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RandomForest RMSE (Val): 0.1475
🏃 View run RandomForest at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0/runs/b07239451908434293e3dc3b97a9479a
🧪 View experiment at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0


In [39]:
rf_params = {"n_estimators": 485, "max_depth": 15}
rf_model = RandomForestRegressor(
    n_estimators=rf_params["n_estimators"],
    max_depth=rf_params["max_depth"],
    random_state=SEED
)
train_and_log(rf_model, "RandomForest", rf_params)

2025/04/09 08:58:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RandomForest RMSE (Val): 0.1481
🏃 View run RandomForest at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0/runs/0deda8dbaa564e2db0f899600e8f9254
🧪 View experiment at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0


In [42]:
# 3. ADVANCED FEATURE ENGINEERING
############################

# 3.1 Example: Remove outliers in GrLivArea if above a threshold
# (common trick for the Ames dataset)
outlier_threshold = 4000
mask = train_df["GrLivArea"] < outlier_threshold
train_df = train_df[mask]  # remove outlier rows


train_df = train_original
test_df = test_original

train_df = fill_missing_values(train_df)
test_df = fill_missing_values(test_df)

train_df["TotalSF"] = train_df["TotalBsmtSF"] + train_df["1stFlrSF"] + train_df["2ndFlrSF"]
test_df["TotalSF"] = test_df["TotalBsmtSF"] + test_df["1stFlrSF"] + test_df["2ndFlrSF"]


# 3.2 Example: Create an interaction feature
# Suppose we already have 'TotalSF' from earlier
# Let's multiply 'OverallQual' by 'TotalSF'
train_df["OverallQual_TotSF"] = train_df["OverallQual"] * (
    train_df["TotalBsmtSF"] + train_df["1stFlrSF"] + train_df["2ndFlrSF"]
)

test_df["OverallQual_TotSF"] = test_df["OverallQual"] * (
    test_df["TotalBsmtSF"] + test_df["1stFlrSF"] + test_df["2ndFlrSF"]
)

# 3.3 Log-transform SalePrice if not already done
train_df["SalePrice"] = np.log1p(train_df["SalePrice"])

# 3.4 Basic drop of ID for train, keep test_ids for submission later
test_ids = test_df["Id"]
train_df.drop(["Id"], axis=1, inplace=True)
test_df.drop(["Id"], axis=1, inplace=True)

# 3.5 Combine train and test for one-hot encoding
all_data = pd.concat([train_df.drop("SalePrice", axis=1), test_df], axis=0)
all_data = pd.get_dummies(all_data, drop_first=True)

# Split back into train vs. test
train_transformed = all_data.iloc[: len(train_df), :]
test_transformed = all_data.iloc[len(train_df) :, :]

X = train_transformed.values
y = train_df["SalePrice"].values  # log-transformed sale price
X_test = test_transformed.values

In [43]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

In [46]:
# 6.1 RandomForest Hyperparam Tuning via GridSearchCV
from xgboost import XGBRegressor
rf = RandomForestRegressor(random_state=SEED)

param_grid_rf = {
    "n_estimators": [100, 200],
    "max_depth": [10, 15, None],
}

grid_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    scoring="neg_mean_squared_error",
    cv=3,
    n_jobs=-1,
    verbose=1
)

print("Fitting RandomForest GridSearch...")
grid_rf.fit(X_train, y_train)

best_params_rf = grid_rf.best_params_
print("Best RandomForest params:", best_params_rf)
best_rf_model = grid_rf.best_estimator_


Fitting RandomForest GridSearch...
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best RandomForest params: {'max_depth': 15, 'n_estimators': 200}


2025/04/09 09:18:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RandomForest_CV RMSE (Val): 0.1475
🏃 View run RandomForest_CV at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0/runs/678dd6908e4d4fcbaa8ba4e0f63ed826
🧪 View experiment at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0


TypeError: cannot unpack non-iterable NoneType object

In [49]:
rf_rmse_val, best_rf_model = train_and_log(
    best_rf_model,
    "RandomForest_CV",
    best_params_rf
)


# 6.2 XGBoost Hyperparam Tuning via GridSearchCV
xgb = XGBRegressor(random_state=SEED, use_label_encoder=False)

param_grid_xgb = {
    "n_estimators": [200, 300],
    "max_depth": [3, 5],
    "learning_rate": [0.05, 0.1],
}

grid_xgb = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid_xgb,
    scoring="neg_mean_squared_error",
    cv=3,
    n_jobs=-1,
    verbose=1
)


2025/04/09 09:20:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RandomForest_CV RMSE (Val): 0.1475
🏃 View run RandomForest_CV at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0/runs/a21694783dc247d5abb7cb4b999b24e4
🧪 View experiment at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0
Fitting XGBoost GridSearch...
Fitting 3 folds for each of 8 candidates, totalling 24 fits


ValueError: 
All the 24 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/xgboost/core.py", line 726, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
TypeError: XGBModel.fit() got an unexpected keyword argument 'eval_metric'


In [50]:

print("Fitting XGBoost GridSearch...")
grid_xgb.fit(X_train, y_train, verbose=False)

best_params_xgb = grid_xgb.best_params_
print("Best XGBoost params:", best_params_xgb)
best_xgb_model = grid_xgb.best_estimator_

# Evaluate best XGB in MLflow
xgb_rmse_val, best_xgb_model = train_and_log(
    best_xgb_model,
    "XGBoost_CV",
    best_params_xgb
)

Fitting XGBoost GridSearch...
Fitting 3 folds for each of 8 candidates, totalling 24 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:23:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best XGBoost params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300}


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:23:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
2025/04/09 09:23:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


XGBoost_CV RMSE (Val): 0.1372
🏃 View run XGBoost_CV at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0/runs/f551859f4f20470aa4e87a1a079fbf65
🧪 View experiment at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0


In [52]:
# 7. ENSEMBLING
# we average predictions from best RF + best XGB
def ensemble_predict(models, weights, X_input):
    """
    Weighted average of predictions from multiple models.
    models: list of fitted models
    weights: same length list of floats
    X_input: feature matrix
    """
    assert len(models) == len(weights), "models and weights must have same length"
    preds = [m.predict(X_input) * w for m, w in zip(models, weights)]
    return np.sum(preds, axis=0) / np.sum(weights)

# Example: let's do a simple uniform average
models_list = [best_rf_model, best_xgb_model]
weights_list = [1.0, 1.0]  # each model is weighted equally

ensemble_preds_val = ensemble_predict(models_list, weights_list, X_val)
ensemble_rmse_val = np.sqrt(mean_squared_error(y_val, ensemble_preds_val))

# Log the ensemble as well
with mlflow.start_run(run_name="Ensemble_RF_XGB"):
    mlflow.log_param("ensemble_method", "average")
    mlflow.log_param("weights", "equal")
    mlflow.log_metric("rmse_val", ensemble_rmse_val)
    print(f"[Ensemble_RF_XGB] RMSE (Val): {ensemble_rmse_val:.4f}")

[Ensemble_RF_XGB] RMSE (Val): 0.1388
🏃 View run Ensemble_RF_XGB at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0/runs/61cb9b36e9214f2b81e20cd88ae92536
🧪 View experiment at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0


In [ ]:

############################
# 8. FINAL CHOICE & SUBMISSION
############################
# Suppose the ensemble did best on validation
# Then we'll fit the ensemble on the entire training set or simply
# use those individual models to predict on X_test and average them.

# Example (simple approach):
ensemble_preds_test = ensemble_predict(models_list, weights_list, X_test)
# Invert the log transform (SalePrice was log1p)
final_preds_test = np.expm1(ensemble_preds_test)

# Create submission dataframe
submission = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": final_preds_test
})